In [1]:
import cv2
import os
import numpy as np
import mediapipe as mp
import torch
from torchvision import transforms
from segment_anything import sam_model_registry, SamPredictor

In [3]:
# Directories
DATA_DIR = "data/outfits"
OUTPUT_DIR = "./processed_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Initialize Pose Estimation (MediaPipe)
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Load Segmentation Model (Segment Anything)
sam_checkpoint = "./models/sam_vit_h.pth"
sam = sam_model_registry["vit_h"](checkpoint=sam_checkpoint)
sam_predictor = SamPredictor(sam)

# Perspective Correction: Define a Reference Posture (Standardized Bounding Box)
REF_WIDTH = 256
REF_HEIGHT = 512
REF_POINTS = np.float32([[50, 50], [200, 50], [50, 450], [200, 450]])  # Reference points for alignment

I0000 00:00:1738529804.710656 9263715 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M3
W0000 00:00:1738529804.844695 9283767 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1738529804.862853 9283761 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [4]:
# Transformation Function
def perspective_correction(image, keypoints):
    if keypoints is None:
        return image  # Skip if no pose detected

    # Select 4 key body landmarks for transformation
    src_points = np.float32([
        keypoints[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
        keypoints[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
        keypoints[mp_pose.PoseLandmark.LEFT_HIP.value],
        keypoints[mp_pose.PoseLandmark.RIGHT_HIP.value]
    ])

    # Compute homography and apply transformation
    matrix = cv2.getPerspectiveTransform(src_points, REF_POINTS)
    corrected_image = cv2.warpPerspective(image, matrix, (REF_WIDTH, REF_HEIGHT))

    return corrected_image

# Segmentation Function
def segment_outfit(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    sam_predictor.set_image(image_rgb)
    masks, _, _ = sam_predictor.predict()
    
    if masks is not None:
        return masks[0]  # Return the first segmentation mask
    return None

In [6]:
# Main Processing Loop
for filename in os.listdir(DATA_DIR):
    if not filename.lower().endswith((".png", ".jpg", ".jpeg")):
        continue
    
    image_path = os.path.join(DATA_DIR, filename)
    image = cv2.imread(image_path)

    if image is None:
        print(f"Warning: Failed to load image {filename}. Skipping...")
        continue  # Skip processing this file

    # Proceed with the rest of the pipeline
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        # Extract Keypoints
        keypoints = [(int(landmark.x * image.shape[1]), int(landmark.y * image.shape[0]))
                     for landmark in results.pose_landmarks.landmark]
        
        # Perspective Correction
        corrected_image = perspective_correction(image, keypoints)

        # Segmentation
        mask = segment_outfit(corrected_image)
        if mask is not None:
            segmented_outfit = cv2.bitwise_and(corrected_image, corrected_image, mask=mask.astype(np.uint8))
        else:
            segmented_outfit = corrected_image

        # Save Processed Image
        output_path = os.path.join(OUTPUT_DIR, filename)
        cv2.imwrite(output_path, segmented_outfit)
        print(f"Processed {filename} -> {output_path}")

print("Preprocessing Complete!")

Processed elegant outfit woman_1.jpg -> ./processed_data/elegant outfit woman_1.jpg
Processed elegant outfit woman_4.jpg -> ./processed_data/elegant outfit woman_4.jpg
Processed elegant outfit woman_5.jpg -> ./processed_data/elegant outfit woman_5.jpg
Processed casual outfit woman_9.jpg -> ./processed_data/casual outfit woman_9.jpg
Processed elegant outfit woman_7.jpg -> ./processed_data/elegant outfit woman_7.jpg
Processed elegant outfit woman_6.jpg -> ./processed_data/elegant outfit woman_6.jpg
Processed casual outfit woman_8.jpg -> ./processed_data/casual outfit woman_8.jpg
Processed elegant outfit woman_10.jpg -> ./processed_data/elegant outfit woman_10.jpg
Processed casual outfit woman_10.jpg -> ./processed_data/casual outfit woman_10.jpg
Processed casual outfit woman_11.jpg -> ./processed_data/casual outfit woman_11.jpg
Processed elegant outfit woman_11.jpg -> ./processed_data/elegant outfit woman_11.jpg
Processed casual outfit woman_13.jpg -> ./processed_data/casual outfit woman